In [ ]:
WANDB_PROJECT = "analysis3"
ENTITY = 'course'
CLASSES = {i:c for i,c in enumerate(['displaced', 'enucleated', 'irregular', 'micronucleus', 'normal', 'other'])}
RAW_DATA_ARTIFACT = 'dataset_23062023'
PROCESSED_DATA_ARTIFACT = 'dataset_23062023_split'

# Configs

In [ ]:
# Installs
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=fbe866202772e91022459626ffd0ba2b8162251d04192461af32549e3a861fa8
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
# Imports
import os
import wandb
import shutil
import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold

# Split

In [ ]:
# Init run and get the artifact
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="data_split", name='split_2')
raw_data_artifact = run.use_artifact(f'{RAW_DATA_ARTIFACT}:latest')
path = Path(raw_data_artifact.download())

In [ ]:
# Get the table and select relevant columns
orig_eda_table = raw_data_artifact.get("eda_table")

fnames = orig_eda_table.get_column('file_name')
groups = orig_eda_table.get_column('original_file')
y = orig_eda_table.get_column('label')

wandb:   2130 of 2130 files downloaded.  


In [ ]:
# Create df for the data
df = pd.DataFrame()
df['file_name'] = fnames
df['fold'] = -1

In [ ]:
# Create folds using stratify partition (by label)
cv = StratifiedKFold(n_splits=10)
for i, (train_idxs, valid_idxs) in enumerate(cv.split(fnames, y)):
    df.loc[valid_idxs, ['fold']] = i

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 8 members, which is less than n_splits=10.
  warnings.warn(


In [ ]:
# Assign two folds for validation (20%)
df['split'] = 'train'
df.loc[df.fold == 0, ['split']] = 'valid'
df.loc[df.fold == 1, ['split']] = 'valid'
del df['fold']
df.split.value_counts()

train    851
valid    214
Name: split, dtype: int64

In [ ]:
# Save df
df.to_csv('data_split.csv', index=False)

In [ ]:
# Create new artifact and add data
processed_data_artifact = wandb.Artifact(PROCESSED_DATA_ARTIFACT, type="split_data")
processed_data_artifact.add_file('data_split.csv')
processed_data_artifact.add_dir(path)

wandb: Adding directory to artifact (./artifacts/dataset_23062023:v0)... Done. 1.1s


In [ ]:
# Create and join table
data_split_table = wandb.Table(dataframe=df[['file_name', 'split']])
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "file_name")
processed_data_artifact.add(join_table, "eda_table_data_split")

In [ ]:
# Log artifact to W&B and finish run
run.log_artifact(processed_data_artifact)
run.finish()